In [1]:
import pickle as pkl
import time
import sys

search=True

from bonsai.data_loaders import load_data
from bonsai.net import Net
from bonsai.trainers import *
from bonsai.helpers import cell_cnx, show_time, namer, TransitionDict, prev_output, wipe_output, mem_stats
from bonsai.ops import commons, Zero

%load_ext autoreload
%autoreload 2

In [2]:
dataset = 'CIFAR10'
classes = 10
gpu_space = 8.4
comp_ratio = 1/4
hypers = {
    'scale':{'init':2,'final':7},
    'batch_size':{'init':256,'final':64},
    'reductions':3,
    'spacing':5,
    'nodes':4,
    'lr_schedule':
        {'lr_min': 0,
         'lr_max': 1e-2,
         't_0': 600,
         't_mult': 1},
    'epochs':600,
    'drop_prob':.25,
}

# Search

In [3]:
if search:
    if 'model' in globals().keys():
        del model
        clean("Search init")
           
    # load date
    batch_size= hypers['batch_size']['init']
    data, data_shape = load_data(batch_size, dataset)
    print("Edges per cell:",cell_cnx(hypers['nodes']))
    
    # init model
    model = Net(dim=data_shape, 
                classes=classes, 
                scale=hypers['scale']['init'],
                reductions=hypers['reductions'], 
                spacing=hypers['spacing'],
                nodes=hypers['nodes'],
                auxiliary=True)
    model.save_genotype()
    size, overflow = size_test(model, data)
    print(model)
    print("Est Size: {}{:.2f}GiB {}".format(">" if overflow else "", size, "(overflow)" if overflow else "")) 
    if overflow:
        del model
        clean('Search init')
    
    # search params
    init_or_finish = True

Edges per cell: 24.0
===================== NETWORK =====================
              Carlton McClure Banbury              
Initializer         :                     20 params
Cell 0  (Normal)    :  4  x 32 ,       2,017 params
Cell 1  (Normal)    :  4  x 32 ,       2,018 params
Cell 2  (Normal)    :  4  x 32 ,       2,019 params
Cell 3  (Normal)    :  4  x 32 ,       2,020 params
Cell 4  (Reduction) :  8  x 32 ,       4,957 params
 ↳ Aux Tower        :                 20,490 params
Cell 5  (Normal)    :  8  x 16 ,       5,078 params
Cell 6  (Normal)    :  8  x 16 ,       5,079 params
Cell 7  (Normal)    :  8  x 16 ,       5,080 params
Cell 8  (Normal)    :  8  x 16 ,       5,081 params
Cell 9  (Reduction) :  16 x 16 ,      13,770 params
 ↳ Aux Tower        :                 10,250 params
Cell 10 (Normal)    :  16 x 8  ,      14,403 params
Cell 11 (Normal)    :  16 x 8  ,      14,404 params
Cell 12 (Normal)    :  16 x 8  ,      14,405 params
Cell 13 (Normal)    :  16 x 8  ,      14,40

In [ ]:
if search:
    prev_output() if not init_or_finish else wipe_output()
    search_start = time.time()
    
    # search parameters
    schedule = {'cycle_len':4,
                'transition_after':2,
                'n_cycles':4}
    epochs = schedule['cycle_len']*schedule['n_cycles']
    transition = schedule['cycle_len']*schedule['transition_after']
    lr_schedule = {'lr_min': hypers['lr_schedule']['lr_min'],
                   'lr_max': hypers['lr_schedule']['lr_max'],
                   't_0': transition,
                   't_mult': 1}    
    # search loop
    for scaling in range(hypers['scale']['init'],hypers['scale']['final']):
        tries = 1
        while 1:
            init_or_finish = full_train(
                model,
                data,
                resume=not init_or_finish,
                epochs=epochs,
                drop_prob=0,
                comp_lambdas=TransitionDict({0: None,
                                             transition: {'edge': .15*tries, 'input': .1*tries}}),
                comp_ratio=comp_ratio,
                prune_interval=schedule['cycle_len'],
                lr_schedule=lr_schedule
            );
            clean(verbose=False)
            size, overflow = size_test(model,data)
            print("Est size: {}, Batch: {}".format(size,batch_size))
            if size > gpu_space/2 and batch_size==hypers['batch_size']['final']:
                comp_ratio*=.75 
                tries+=1
                print("Restarting pruning at scale level {}, new comp ratio: {}".format(scaling,comp_ratio))
            elif size>(gpu_space/2) and batch_size>hypers['batch_size']['final']:
                batch_size=batch_size//2
                data, data_shape  = load_data(batch_size,dataset)
                print("Lowering batch size to {} for scaling".format(batch_size))
                break
            else:
                break   

        if init_or_finish:
            model.save_genotype()
            clean("Prescale")
            model.scale_up()
            model_id = model.model_id
        else:
            clean()
            break
    print("Search Time:",show_time(time.time()-search_start))

=== Training Carlton McClure Banbury ===
0: 22.00MiB
1: 862.00MiB
2: 1.60GiB
3: 2.37GiB
4: 3.13GiB
5: 3.54GiB
6: 3.99GiB
7: 4.44GiB
8: 4.88GiB
9: 5.33GiB
10: 5.62GiB
11: 5.93GiB
12: 6.24GiB
13: 6.56GiB
14: 6.88GiB
Train Epoch: 0   [15680 /50000  (100%)]	Loss: 2.384, Losses [0: 1.54, 1: 1.47, 2: 1.52, 3: 1.48]Per Epoch: 3m,25s , Alloc: 7.11GiB  
Train Corrects: Top-1: 35.39%, 4m,41s
Test  Corrects: Top-1: 43.48%, 14.57 s

Adjusting lr to 0.01
Train Epoch: 1   [15680 /50000  (100%)]	Loss: 2.351, Losses [0: 1.56, 1: 1.46, 2: 1.5, 3: 1.45]Per Epoch: 3m,44s , Alloc: 7.11GiB  
Train Corrects: Top-1: 44.17%, 4m,40s
Test  Corrects: Top-1: 44.88%, 14.52 s

Adjusting lr to 0.009619397662556433
Train Epoch: 2   [15680 /50000  (100%)]	Loss: 2.228, Losses [0: 1.52, 1: 1.47, 2: 1.35, 3: 1.36]Per Epoch: 3m,40s , Alloc: 7.11GiB  
Train Corrects: Top-1: 47.45%, 4m,37s
Test  Corrects: Top-1: 49.06%, 14.23 s

Adjusting lr to 0.008535533905932738
Train Epoch: 3   [15680 /50000  (100%)]	Loss: 2.167, Losses

Train Epoch: 12  [49984 /50000  (100%)]	Comp Ratio: [E: 0.455, I: 1.800], Loss Comp: [C: 1.297, E: 0.126, I: 0.07], Losses [0: 1.27, 1: 0.69, 2: 0.55, 3: 0.59]Per Epoch: 11m,26s, Alloc: 4.92GiB  
Train Corrects: Top-1: 78.74%, Comp: 0.45, 1.73 11m,56s
Train Loss Components: C: 1.337, E: 0.126, I: 0.07
Test  Corrects: Top-1: 74.08%, 30.55 s

Adjusting lr to 0.005
Train Epoch: 13  [49984 /50000  (100%)]	Comp Ratio: [E: 0.456, I: 1.933], Loss Comp: [C: 1.217, E: 0.128, I: 0.08], Losses [0: 1.4, 1: 0.65, 2: 0.51, 3: 0.5]Per Epoch: 11m,45s, Alloc: 4.92GiB   
Train Corrects: Top-1: 80.77%, Comp: 0.46, 1.93 11m,57s
Train Loss Components: C: 1.142, E: 0.127, I: 0.08
Test  Corrects: Top-1: 80.78%, 30.66 s

Adjusting lr to 0.0030865828381745515
Train Epoch: 14  [49984 /50000  (100%)]	Comp Ratio: [E: 0.450, I: 1.733], Loss Comp: [C: 1.235, E: 0.123, I: 0.07], Losses [0: 1.17, 1: 0.74, 2: 0.56, 3: 0.54]Per Epoch: 11m,23s, Alloc: 4.92GiB  
Train Corrects: Top-1: 82.86%, Comp: 0.45, 1.73 11m,57s
Tra

Train Epoch: 1   [49984 /50000  (100%)]	Loss: 2.562, Losses [0: 1.98, 1: 1.14, 2: 0.7, 3: 1.8]Per Epoch: 7m,18s , Alloc: 7.89GiB   
Train Corrects: Top-1: 70.46%, 7m,42s
Test  Corrects: Top-1: 60.38%, 25.47 s

Adjusting lr to 0.009619397662556433
Train Epoch: 2   [49984 /50000  (100%)]	Loss: 5.244, Losses [0: 2.08, 1: 1.43, 2: 1.22, 3: 4.3]Per Epoch: 7m,29s , Alloc: 7.89GiB  
Train Corrects: Top-1: 68.31%, 7m,42s
Test  Corrects: Top-1: 62.32%, 25.89 s

Adjusting lr to 0.008535533905932738
Train Epoch: 3   [49984 /50000  (100%)]	Loss: 3.510, Losses [0: 2.17, 1: 1.13, 2: 0.86, 3: 2.68]Per Epoch: 7m,39s , Alloc: 7.89GiB  
Train Corrects: Top-1: 62.27%, 7m,41s

Deadheaded 3 operations
Param Delta: 3,940,501 -> 3,937,746
Pre-prune Test  Corrects: Top-1: 69.10%, 25.27 s
Post-prune Test  Corrects: Top-1: 69.10%, 25.38 s

Adjusting lr to 0.00691341716182545
Train Epoch: 4   [49984 /50000  (100%)]	Loss: 2.701, Losses [0: 2.55, 1: 1.58, 2: 0.76, 3: 1.72]Per Epoch: 8m,11s , Alloc: 7.89GiB  
Train

Test  Corrects: Top-1: 87.17%, 22.95 s

Restarting Learning Rate, setting new cycle length to 8

Adjusting lr to 0.01
Train Epoch: 9   [49984 /50000  (100%)]	Comp Ratio: [E: 0.342, I: 1.533], Loss Comp: [C: 2.076, E: 0.372, I: 0.19], Losses [0: 1.62, 1: 0.96, 2: 0.62, 3: 0.88]Per Epoch: 9m,23s , Alloc: 6.93GiB  
Train Corrects: Top-1: 81.24%, Comp: 0.34, 1.53 9m,27s
Train Loss Components: C: 4.062, E: 0.369, I: 0.19
Test  Corrects: Top-1: 76.51%, 23.29 s

Adjusting lr to 0.009619397662556433
Train Epoch: 10  [49984 /50000  (100%)]	Comp Ratio: [E: 0.336, I: 1.333], Loss Comp: [C: 1.579, E: 0.359, I: 0.17], Losses [0: 1.89, 1: 1.11, 2: 0.39, 3: 0.37]Per Epoch: 9m,3s  , Alloc: 6.93GiB  
Train Corrects: Top-1: 78.60%, Comp: 0.33, 1.33 9m,27s
Train Loss Components: C: 1.293, E: 0.357, I: 0.17
Test  Corrects: Top-1: 78.85%, 22.97 s

Adjusting lr to 0.008535533905932738
Train Epoch: 11  [49984 /50000  (100%)]	Comp Ratio: [E: 0.330, I: 1.400], Loss Comp: [C: 2.958, E: 0.350, I: 0.17], Losses [

Post-prune Test  Corrects: Top-1: 83.21%, 19.65 s

Adjusting lr to 0.00691341716182545
Train Epoch: 4   [49984 /50000  (100%)]	Loss: 0.962, Losses [0: 1.21, 1: 0.68, 2: 0.43, 3: 0.5]Per Epoch: 5m,24s , Alloc: 5.66GiB  
Train Corrects: Top-1: 83.34%, 5m,39s
Test  Corrects: Top-1: 85.07%, 19.45 s

Adjusting lr to 0.005
Train Epoch: 5   [49984 /50000  (100%)]	Loss: 0.865, Losses [0: 1.36, 1: 0.61, 2: 0.34, 3: 0.4]Per Epoch: 5m,34s , Alloc: 5.66GiB  
Train Corrects: Top-1: 85.98%, 5m,39s
Test  Corrects: Top-1: 86.23%, 19.68 s

Adjusting lr to 0.0030865828381745515
Train Epoch: 6   [49984 /50000  (100%)]	Loss: 0.616, Losses [0: 0.88, 1: 0.58, 2: 0.29, 3: 0.27]Per Epoch: 5m,16s , Alloc: 5.66GiB  
Train Corrects: Top-1: 87.80%, 5m,40s
Test  Corrects: Top-1: 88.03%, 19.58 s

Adjusting lr to 0.0014644660940672626
Train Epoch: 7   [49984 /50000  (100%)]	Loss: 0.593, Losses [0: 0.75, 1: 0.48, 2: 0.32, 3: 0.28]Per Epoch: 5m,43s , Alloc: 5.66GiB  
Train Corrects: Top-1: 89.01%, 5m,39s

Deadheaded 0

...

Train Epoch: 2   [49984 /50000  (100%)]	Loss: 1.763, Losses [0: 2.79, 1: 1.27, 2: 0.53, 3: 0.85]Per Epoch: 3m,11s , Alloc: 5.23GiB  
Train Corrects: Top-1: 78.92%, 3m,29s
Test  Corrects: Top-1: 76.63%, 13.32 s

Adjusting lr to 0.008535533905932738
Train Epoch: 3   [49984 /50000  (100%)]	Loss: 1.664, Losses [0: 2.02, 1: 0.73, 2: 0.51, 3: 1.01]Per Epoch: 3m,15s , Alloc: 5.23GiB  
Train Corrects: Top-1: 81.29%, 3m,29s

Deadheaded 0 operations
Param Delta: 2,998,791 -> 2,998,791
Pre-prune Test  Corrects: Top-1: 81.37%, 13.34 s
Post-prune Test  Corrects: Top-1: 81.37%, 13.34 s

Adjusting lr to 0.00691341716182545
Train Epoch: 4   [49984 /50000  (100%)]	Loss: 0.704, Losses [0: 1.14, 1: 0.66, 2: 0.28, 3: 0.29]Per Epoch: 3m,21s , Alloc: 5.23GiB  
Train Corrects: Top-1: 83.54%, 3m,29s
Test  Corrects: Top-1: 82.82%, 13.32 s

Adjusting lr to 0.005
Train Epoch: 5   [49984 /50000  (100%)]	Loss: 0.743, Losses [0: 1.58, 1: 0.53, 2: 0.24, 3: 0.27]Per Epoch: 3m,27s , Alloc: 5.23GiB  
Train Corrects: To

Train Epoch: 14  [49984 /50000  (100%)]	Comp Ratio: [E: 0.124, I: 1.400], Loss Comp: [C: 5.417, E: 3.117, I: 1.62], Losses [0: 0.79, 1: 0.43, 2: 0.34, 3: 0.37]Per Epoch: 4m,47s , Alloc: 5.23GiB  
Train Corrects: Top-1: 88.83%, Comp: 0.12, 1.00 4m,56s
Train Loss Components: C: 5.682, E: 3.117, I: 0.47
Test  Corrects: Top-1: 88.82%, 13.24 s

Adjusting lr to 0.0014644660940672626
Train Epoch: 15  [49984 /50000  (100%)]	Comp Ratio: [E: 0.124, I: 1.400], Loss Comp: [C: 5.135, E: 3.117, I: 1.62], Losses [0: 0.88, 1: 0.28, 2: 0.17, 3: 0.13]Per Epoch: 5m,19s , Alloc: 5.23GiB  
Train Corrects: Top-1: 89.58%, Comp: 0.12, 1.40 4m,57s
Train Loss Components: C: 6.343, E: 3.117, I: 1.62

Deadheaded 1 operations
Param Delta: 2,996,230 -> 2,960,389
Pre-prune Test  Corrects: Top-1: 89.26%, 13.27 s
Post-prune Test  Corrects: Top-1: 89.26%, 13.17 s

Adjusting lr to 0.0003806023374435663
Est size: 4.974609375, Batch: 64
Restarting pruning at scale level 6, new comp ratio: 2.5169851396752785e-07
=== Traini

Post-prune Test  Corrects: Top-1: 69.16%, 12.95 s

Adjusting lr to 0.00691341716182545
Train Epoch: 4   [49984 /50000  (100%)]	Loss: 1.359, Losses [0: 1.93, 1: 0.88, 2: 0.45, 3: 0.71]Per Epoch: 3m,11s , Alloc: 5.05GiB  
Train Corrects: Top-1: 73.70%, 3m,22s
Test  Corrects: Top-1: 68.51%, 12.95 s

Adjusting lr to 0.005
Train Epoch: 5   [49984 /50000  (100%)]	Loss: 1.636, Losses [0: 1.71, 1: 0.77, 2: 0.58, 3: 1.02]Per Epoch: 3m,6s  , Alloc: 5.05GiB  
Train Corrects: Top-1: 78.06%, 3m,22s
Test  Corrects: Top-1: 78.27%, 12.95 s

Adjusting lr to 0.0030865828381745515
Train Epoch: 6   [49984 /50000  (100%)]	Loss: 1.243, Losses [0: 2.15, 1: 0.86, 2: 0.53, 3: 0.54]Per Epoch: 3m,7s  , Alloc: 5.05GiB  
Train Corrects: Top-1: 81.16%, 3m,22s
Test  Corrects: Top-1: 80.26%, 12.95 s

Adjusting lr to 0.0014644660940672626
Train Epoch: 7   [49984 /50000  (100%)]	Loss: 1.401, Losses [0: 1.57, 1: 0.99, 2: 0.57, 3: 0.78]Per Epoch: 3m,7s  , Alloc: 5.05GiB  
Train Corrects: Top-1: 82.63%, 3m,21s

Deadheaded

Train Epoch: 15  [49984 /50000  (100%)]	Comp Ratio: [E: 0.121, I: 0.933], Loss Comp: [C: 5.527, E: 3.368, I: 0.35], Losses [0: 2.24, 1: 1.07, 2: 0.89, 3: 0.97]Per Epoch: 4m,48s , Alloc: 5.05GiB  
Train Corrects: Top-1: 82.28%, Comp: 0.12, 1.33 4m,48s
Train Loss Components: C: 5.584, E: 3.368, I: 1.75

Deadheaded 1 operations
Param Delta: 2,950,660 -> 2,813,443
Pre-prune Test  Corrects: Top-1: 84.16%, 12.93 s
Post-prune Test  Corrects: Top-1: 84.16%, 12.83 s

Adjusting lr to 0.0003806023374435663
Est size: 4.8203125, Batch: 64
Restarting pruning at scale level 6, new comp ratio: 7.96389829350381e-08
=== Training Carlton McClure Banbury ===
0: 76.00MiB
1: 860.00MiB
2: 1.51GiB
3: 2.23GiB
4: 3.09GiB
5: 3.67GiB
6: 3.77GiB
7: 3.88GiB
8: 4.15GiB
9: 4.38GiB
10: 4.45GiB
11: 4.45GiB
12: 4.45GiB
13: 4.59GiB
14: 4.71GiB
Train Epoch: 0   [49984 /50000  (100%)]	Loss: 2.974, Losses [0: 3.79, 1: 1.58, 2: 0.9, 3: 1.72]Per Epoch: 3m,9s  , Alloc: 4.88GiB    
Train Corrects: Top-1: 69.12%, 3m,20s
Test  Co

Train Epoch: 11  [49984 /50000  (100%)]	Comp Ratio: [E: 0.119, I: 0.867], Loss Comp: [C: 5.437, E: 3.539, I: 0.55], Losses [0: 1.89, 1: 0.86, 2: 0.57, 3: 0.68]Per Epoch: 4m,36s , Alloc: 4.98GiB  
Train Corrects: Top-1: 75.39%, Comp: 0.12, 0.87 4m,46s
Train Loss Components: C: 6.979, E: 3.582, I: 0.55

Deadheaded 0 operations
Param Delta: 2,812,930 -> 2,812,930
Pre-prune Test  Corrects: Top-1: 80.45%, 12.82 s
Post-prune Test  Corrects: Top-1: 80.45%, 12.82 s

Adjusting lr to 0.00691341716182545
Train Epoch: 12  [49984 /50000  (100%)]	Comp Ratio: [E: 0.120, I: 1.267], Loss Comp: [C: 7.030, E: 3.582, I: 1.92], Losses [0: 2.36, 1: 1.06, 2: 0.55, 3: 0.74]Per Epoch: 4m,42s , Alloc: 4.98GiB  
Train Corrects: Top-1: 77.50%, Comp: 0.12, 1.73 4m,51s
Train Loss Components: C: 8.375, E: 3.582, I: 3.09
Test  Corrects: Top-1: 80.83%, 13.59 s

Adjusting lr to 0.005
Train Epoch: 13  [49984 /50000  (100%)]	Comp Ratio: [E: 0.119, I: 1.267], Loss Comp: [C: 6.844, E: 3.531, I: 1.92], Losses [0: 1.39, 1: 0

# Train

In [ ]:
data, data_shape = load_data(hypers['batch_size']['final'], dataset)
params, genotype = pkl.load(open('genotypes/genotype_{}.pkl'.format(name.replace(" ","_")),'rb'))

params['genotype']=genotype
params['scale']=hypers['scale']['final']
params['prune']=False
params['auxiliary']=True
params['dim']=data_shape
model = Net(**params)
print(model)

init_or_finish = True
size_test(model,data)
actual_comp_ratio = model.genotype_compression()

In [ ]:
prev_output() if not init_or_finish else wipe_output()
init_or_finish = full_train(
    model, data,
    lr_schedule = hypers['lr_schedule'],
    resume=not init_or_finish,
    epochs=hypers['epochs'],
    drop_prob=hypers['drop_prob'],
);
clean()

# Random Search

In [ ]:
data,data_shape = load_data(hypers['batch_size']['final'], dataset)
model = Net(dim=data_shape, 
            classes=classes, 
            scale=hypers['scale']['final'],
            reductions=hypers['reductions'], 
            spacing=hypers['spacing'],
            nodes=hypers['nodes'],
            random_ops=actual_comp_ratio, 
            prune=False,
            auxiliary=True)
model.save_genotype()
print(model)
print(size_test(model,data))
model.genotype_compression()

In [ ]:
full_train(
    model, data,
    lr_schedule = hypers['lr_schedule'],
    epochs=hypers['epochs'],
    drop_prob=hypers['drop_prob'],
);